In [3]:
import pandas as pd

data = pd.read_excel("solar_data.xlsx", header=3)
columns = data.columns

False

In [14]:
list(columns[4:])

['Jan 2019 - SGU Rated Output In kW',
 'Feb 2019 - Installations Quantity',
 'Feb 2019 - SGU Rated Output In kW',
 'Mar 2019 - Installations Quantity',
 'Mar 2019 - SGU Rated Output In kW',
 'Apr 2019 - Installations Quantity',
 'Apr 2019 - SGU Rated Output In kW',
 'May 2019 - Installations Quantity',
 'May 2019 - SGU Rated Output In kW',
 'Jun 2019 - Installations Quantity',
 'Jun 2019 - SGU Rated Output In kW',
 'Jul 2019 - Installations Quantity',
 'Jul 2019 - SGU Rated Output In kW',
 'Aug 2019 - Installations Quantity',
 'Aug 2019 - SGU Rated Output In kW',
 'Sep 2019 - Installations Quantity',
 'Sep 2019 - SGU Rated Output In kW',
 'Oct 2019 - Installations Quantity',
 'Oct 2019 - SGU Rated Output In kW',
 'Nov 2019 - Installations Quantity',
 'Nov 2019 - SGU Rated Output In kW',
 'Dec 2019 - Installations Quantity',
 'Dec 2019 - SGU Rated Output In kW',
 'Jan 2020 - Installations Quantity',
 'Jan 2020 - SGU Rated Output In kW',
 'Feb 2020 - Installations Quantity',
 'Feb 2020 -